**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [63]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
import matplotlib.pyplot as plt

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Flatten,Softmax

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [64]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [65]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    
    def set_epsilon(self,e):
        self.epsilon = e

        
    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        return 0

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function $act$ is divided in 2 use-cases : train or not. In the 2 cases, it uses the last state to compute the next action according to the policy. When the model is training, epsilon is here to avoid the model to be stuck on a stationary point. It is essential for the training to go as far as possible. When the model is not training, we want to use only the learned policy to perform the predictions, since the epsilon would drop the accuracy, or any other measure of test performance.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [66]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        #used to display the game
        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        """displaying the game"""
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        """generates the frame of the game to display at the time t"""
        
        #the scenery is grey
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        
        #the cheeses are shown to be green
        b[self.board>0,0] = 256
        
        #the poisons are shown to be red
        b[self.board < 0, 2] = 256
        
        #displays a white cell to show the position of the rat
        b[self.x,self.y,:]=256
        
        #plugs the contours according to the 2-cases vision of the rat
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        #resize the frame for it to be displayed correctly
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        #upgrades the final video of the game
        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""
        
        #generate the frame corresponding to the current state of the game
        self.get_frame(int(self.t))

        #matrix of the admissible positions for the rat
        self.position = np.zeros((self.grid_size, self.grid_size))

        #the rat can't go up to 2 cases off the boundaries
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        
        #the rat current position
        self.position[self.x, self.y] = 1
        
        #in function of the chosen action (up,down,right,left), changes the position of the rat
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        #udpates the time of the games
        self.t = self.t + 1
        
        #computes the reward according to the environment
        reward = self.board[self.x, self.y]
        
        #udpates the fact that the rat as changed position
        self.board[self.x, self.y] = 0
        
        #game is over if the game is running out of time
        game_over = self.t > self.max_time
        
        #gets the information about the objects the rat sees
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        #the position of the rat is initialised at a random valid position (off the poisonous cells)
        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        #randomly add bonus on the board, with a density self.temperature
        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        #randomly add malus on the board, with a density self.temperature
        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)
        
        #initialises the video of the game to be displayed
        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        #removes the maluses which overlap the bonus
        malus[bonus>0]=0
        
        #plug the bonus and malus on the board
        self.board = bonus + malus

        #matrix of positions of the rat
        self.position = np.zeros((self.grid_size, self.grid_size))
        
        #boundaries (the rat can not go there)
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        
        #initial position of the rat
        self.board[self.x,self.y] = 0
        
        #initialises time
        self.t = 0

        #the state is composed of the poisonous cell, cheese cells and boundary cells the rat sees
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [67]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train = 30 # set small when debugging
epochs_test  = 10  # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array $position$ is given to have information the rat position on the grid, and also the information of the position of the boundary cells, so that the rat does not go out of the grid.
The array $board$ is used to know where are the objects (cheese and poison) on the grid.
The two arrays concatenated are all the rat needs to know to play the game. 

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [68]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    
    def learned_act(self, s):
        """random agent always choose to perform a random action"""
        return np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [69]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
    
    state = env.reset
    
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state,train=False) #we are in test mode

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(e)

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epochs, loss, win, lose, win-lose))
        agent.save()
        
        # Save as a mp4
        env.draw(prefix+
                 str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [53]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

<class 'int'>
<class 'int'>
<class 'int'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'int'>
Epoch 000/010 | Loss 0.0000 | Win/lose count 5.5/8.0 (-2.5)
Win/lose count 5.5/8.0. Average score (-5.0)
<class 'int'>
<class 'int'>
<class 'int'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'int'>
Epoch 001/010 | Loss 0.0000 | Win/lose count 9.5/7.0 (2.5)
Win/lose count 9.5/7.0. Average score (0.0)
<class 'int'>
<class 'int'>
<class 'int'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'int'>
Epoch 002/010 | Loss 0.0000 | Win/lose count 10.5/6.0 (4.5)
Win/lose count 10.5/6.0. Average score (3.0)
<class 'int'>
<class 'int'>
<class 'int'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'int'>
Epoch 003/010 | Loss 0.0000 | Win/lose count 8.5/13.0 (-4.5)
Win/lose count 8.5/13.0. Average score (0.0)
<class 'int'>
<class 'int'>
<class 'int'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'int'>
Epoch 004/010 | Loss 0.0000 | Win/lose count 9.5/13.0 (-3.5)


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}


\begin{eqnarray}
Q^{\pi}(s,a) &=& E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\nonumber \\
&=& E_{p^\pi} \left[\displaystyle r(a_0,s_0) + \gamma \sum_{t=1}^\infty \gamma^{t-1} r(a_t,s_t) \left|a_0 = a, s_0 = s\right.\right]\nonumber\\
&=& r(a_0,s_0) + \gamma E_{p^{\pi}}\left[\sum_{t=0}^{\infty} \gamma^{t} r(a_{t+1},s_{t+1}) \left|a_0 = a, s_0 = s\right.\right]\nonumber\\
&=& r(a,s) + \gamma E_{(a',s') \sim p^\pi(.|a,s)} \left[E_{p^{\pi}}\left[\sum_{t=0}^{\infty} \gamma^{t} r(a_{t+1},s_{t+1}) \left|a_1 = a', s_1 = s'\right.\right]\right]\text{ (Total expectation)}\nonumber\\
&=& r(a,s) + \gamma E_{(a',s') \sim p^\pi(.|a,s)} \left[E_{p^{\pi}}\left[\sum_{t=0}^{\infty} \gamma^{t} r(a_{t},s_{t}) \left|a_0 = a', s_0 = s'\right.\right]\right]\nonumber\\
&=& E_{(a',s') \sim p^\pi(.|a,s)} \left[\displaystyle r(a,s) + \gamma Q^{\pi}(s',a') \right]
\end{eqnarray}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

Since the optimal policy always chooses the best possible action, taking the expectation over the variable $a'$ boils down to maximizing over this same variable.
Hence, we have :
\begin{eqnarray}
 Q^{*}(s,a) &=& \underset{\pi}{\max}\text{ }\underset{a'}{\max}\text{ }E_{s' \sim p^\pi(.|a,s)} \left[\displaystyle r(a,s) + \gamma Q^{\pi}(s',a') \right]\nonumber\\
 &=& \underset{\pi}{\max}\text{ }E_{s' \sim p^\pi(.|a,s)} \left[\displaystyle r(a,s) + \gamma \underset{a'}{\max}\text{ }Q^{\pi}(s',a') \right]\nonumber\\
 &=& \text{ }E_{s' \sim \pi^*(.|a,s)} \left[\displaystyle r(a,s) + \gamma \underset{a'}{\max}\text{ }Q^{\pi^*}(s',a') \right]
\end{eqnarray}

Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

We want :
\begin{eqnarray}
 Q^{*}(s,a) &=& E_{s' \sim \pi^*(.|a,s)} \left[\displaystyle r(a,s) + \gamma \underset{a'}{\max}\text{ }Q^{\pi^*}(s',a') \right]\nonumber\\
 \Leftrightarrow Q^{*}(s,a) &-& E_{s' \sim \pi^*(.|a,s)} \left[\displaystyle r(a,s) + \gamma \underset{a'}{\max}\text{ }Q^{\pi^*}(s',a') \right] = 0
\end{eqnarray}

Hence, we want to minimize $Q^{*}(s,a) - E_{s' \sim \pi^*(.|a,s)} \left[\displaystyle r(a,s) + \gamma \underset{a'}{\max}\text{ }Q^{\pi^*}(s',a') \right]$, so a plausible objective function is :
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [70]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

        
    def remember(self, m):
        """takes the last parameters generated by the agent and store them in the memory"""
        if len(self.memory) <= self.max_memory:
            self.memory.append(m)
        else :
            #if the size of the memory is too large, it removes the first elements (it forgets them)
            del self.memory[0]
            self.memory.append(m)

    def random_access(self):
        """provides random access to the elements stored in the memory"""
        random_time = np.random.randint(len(self.memory))
        return random_time,self.memory[random_time]

***
The pipeline we will use for training is given below:

In [71]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [92]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        # function wich computes the best action for the agent to perform by maximising Q over the action space
        
        #computing Q using the neural network
        Q = self.model.predict(s.reshape((1,5,5,self.n_state)))
        
        return np.argmax(Q)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            
            #pick a random set of state-action-new_state from the memory to compute the loss on it
            time_random, memory_random = self.memory.random_access()
            
            #fills the input states as the state and new state
            input_states[i,:,:,:] = memory_random[0]
            
            if game_over_:
                #there is no next state so the computation of Q only takes the current state into account
                target_q[i,:] = self.model.predict(s_.reshape((1,5,5,self.n_state)))
            else:
                #computes Q without looking at the next state
                target_q[i,:] = self.model.predict(s_.reshape((1,5,5,self.n_state)))
            
                #computing the posterior value of Q looking at the next state, for the action which has been done,
                #using the formula established above
                target_q[i,a_] = r_ + self.discount*np.max(self.model.predict(n_s_.reshape((1,5,5,self.n_state))))


        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model : it is used to compute the Q function for state 
                
        model = Sequential()

        #reshaping the input states in a 1-dimensional vector
        model.add(Reshape((5*5*self.n_state,), input_shape=(5,5,self.n_state)))
        
        #projection onto a higher-dimensionnal space (often used with fully-connected layers)
        model.add(Dense(2*5*5*self.n_state))
        model.add(BatchNormalization())
        
        #first layer
        model.add(Dense(16))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        #last layer : computation of Q for every possible action
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [80]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 128)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train0.mp4'))

Epoch 000/030 | Loss 0.0122 | Win/lose count 4.5/5.0 (-0.5)
Epoch 001/030 | Loss 0.0146 | Win/lose count 4.5/14.0 (-9.5)
Epoch 002/030 | Loss 0.0072 | Win/lose count 4.0/9.0 (-5.0)
Epoch 003/030 | Loss 0.0132 | Win/lose count 4.0/4.0 (0.0)
Epoch 004/030 | Loss 0.0060 | Win/lose count 2.5/8.0 (-5.5)
Epoch 005/030 | Loss 0.0094 | Win/lose count 9.0/9.0 (0.0)
Epoch 006/030 | Loss 0.0032 | Win/lose count 7.0/7.0 (0.0)
Epoch 007/030 | Loss 0.0027 | Win/lose count 4.0/5.0 (-1.0)
Epoch 008/030 | Loss 0.0034 | Win/lose count 3.5/12.0 (-8.5)
Epoch 009/030 | Loss 0.0047 | Win/lose count 2.0/4.0 (-2.0)
Epoch 010/030 | Loss 0.0014 | Win/lose count 7.0/14.0 (-7.0)
Epoch 011/030 | Loss 0.0007 | Win/lose count 5.0/4.0 (1.0)
Epoch 012/030 | Loss 0.0009 | Win/lose count 6.0/7.0 (-1.0)
Epoch 013/030 | Loss 0.0008 | Win/lose count 2.0/4.0 (-2.0)
Epoch 014/030 | Loss 0.0021 | Win/lose count 3.0/7.0 (-4.0)
Epoch 015/030 | Loss 0.0005 | Win/lose count 3.5/8.0 (-4.5)
Epoch 016/030 | Loss 0.0012 | Win/lose co

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [93]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        # NN Model : it is used to compute the Q function for state
                
        model = Sequential()
        
        #first layer of computing Q
        model.add(Conv2D(filters=1,kernel_size = (3,3)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        #second layer of computing Q
        model.add(Conv2D(filters=1,kernel_size = (3,3)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(Flatten())
        
        #last layer is a fully-connected : computation of Q for every possible action
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [82]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 128)
#for layer in agent.model.layers:
#    print(layer.output_shape)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train0.mp4'))

Epoch 000/030 | Loss 0.0000 | Win/lose count 5.5/8.0 (-2.5)
Epoch 001/030 | Loss 0.0000 | Win/lose count 1.5/6.0 (-4.5)
Epoch 002/030 | Loss 0.0000 | Win/lose count 2.5/2.0 (0.5)
Epoch 003/030 | Loss 0.0000 | Win/lose count 1.5/1.0 (0.5)
Epoch 004/030 | Loss 0.0000 | Win/lose count 4.0/3.0 (1.0)
Epoch 005/030 | Loss 0.0000 | Win/lose count 5.5/6.0 (-0.5)
Epoch 006/030 | Loss 0.0000 | Win/lose count 3.5/5.0 (-1.5)
Epoch 007/030 | Loss 0.0000 | Win/lose count 2.5/1.0 (1.5)
Epoch 008/030 | Loss 0.0000 | Win/lose count 4.0/1.0 (3.0)
Epoch 009/030 | Loss 0.0000 | Win/lose count 1.0/2.0 (-1.0)
Epoch 010/030 | Loss 0.0000 | Win/lose count 6.0/11.0 (-5.0)
Epoch 011/030 | Loss 0.0000 | Win/lose count 4.0/9.0 (-5.0)
Epoch 012/030 | Loss 0.0000 | Win/lose count 5.5/4.0 (1.5)
Epoch 013/030 | Loss 0.0000 | Win/lose count 4.0/3.0 (1.0)
Epoch 014/030 | Loss 0.0000 | Win/lose count 2.5/3.0 (-0.5)
Epoch 015/030 | Loss 0.0000 | Win/lose count 2.0/4.0 (-2.0)
Epoch 016/030 | Loss 0.0000 | Win/lose count 3

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [83]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 128)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 128)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Epoch 000/010 | Loss 0.0024 | Win/lose count 1.0/0 (1.0)
Win/lose count 1.0/0. Average score (2.0)
Epoch 001/010 | Loss 0.0008 | Win/lose count 0.5/5.0 (-4.5)
Win/lose count 0.5/5.0. Average score (-3.5)
Epoch 002/010 | Loss 0.0003 | Win/lose count 1.0/2.0 (-1.0)
Win/lose count 1.0/2.0. Average score (-3.0)
Epoch 003/010 | Loss 0.0004 | Win/lose count 2.5/1.0 (1.5)
Win/lose count 2.5/1.0. Average score (-1.5)
Epoch 004/010 | Loss 0.0007 | Win/lose count 1.0/1.0 (0.0)
Win/lose count 1.0/1.0. Average score (-1.2)
Epoch 005/010 | Loss 0.0004 | Win/lose count 1.5/2.0 (-0.5)
Win/lose count 1.5/2.0. Average score (-1.1666666666666667)
Epoch 006/010 | Loss 0.0026 | Win/lose count 1.5/3.0 (-1.5)
Win/lose count 1.5/3.0. Average score (-1.4285714285714286)
Epoch 007/010 | Loss 0.0006 | Win/lose count 0.5/2.0 (-1.5)
Win/lose count 0.5/2.0. Average score (-1.625)
Epoch 008/010 | Loss 0.0003 | Win/lose count 2.0/6.0 (-4.0)
Win/lose count 2.0/6.0. Average score (-2.3333333333333335)


In [84]:
HTML(display_videos('cnn_test0.mp4'))

In [85]:
HTML(display_videos('fc_test0.mp4'))

Both algorithms improve the final score in comparison with the one performed by the Random Agent. We can notice that for the same number of epochs, the CNN performs better : it might get faster spatial information since it has less coefficients and convolution being a good way to propagate spatial information.

The agent tends to be blocked as it learns : in fact, when it has picked all the bonuses in an area surrounded by poison, it is stuck because there is no evident action to perform for him to maximize Q. This situation is more likely to happen as the temperature grows, since there is more poison and then more chances to be stuck in this kind of situation.

Mathematically speaking, the agents gets stuck on a local maxima of its policy : since there is poison around it, the expected reward is very low and even if it is on cells where the reward is zero, it will choose to stick here since it has no better option. This problem may be solve by rising the vision field of the agent, but I did not understand if it is really a rule of the game, or merely a limitation in order not to have too much data to process.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.

After implementing the decreasing $\epsilon$-greedy exploration and having adapted the class Environment by incorporating a new state, we can see that the results are better (systematically above 0, although it was always below zero without perform these modifications).

In [102]:
def train_explore(agent,env,epoch,prefix='',epsilon_final= 0.1):
    # Number of won games
    score = 0
    loss = 0
    
    beta_epsilon = epsilon_final**(1/env.max_time)
    
    #at first, sets the policy on random
    agent.set_epsilon(1)

    for e in range(epoch):
        
        #at each step, adjusts the balance between Random Agent and Learn Agent
        agent.set_epsilon(agent.epsilon*beta_epsilon)
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(Environment):
    #def __init__(self, grid_size=10, max_time=500, temperature=0.1):
    def __init__(self):
        super(EnvironmentExploring, self).__init__()
        
        malus = 0.4
        
        #Adding a new state corresponding the cells already explored by the agent
        #initialises it all to zero, and to one for the cell on which agent begins
        self.visited_cells = np.zeros((self.grid_size, self.grid_size))
        self.visited_cells[self.x, self.y] = -malus #the value is arbitrary, it is designed to be a lot less than poisonnous cells
        #but still huge enough to be taken into account by the agent

    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""
        state, reward, game_over = Environment.act(self, action)
        
        malus = 0.4
        
        if train:
            # adds the information of the agent having already went through the cell
            reward += self.visited_cells[self.x, self.y]
        
        #if the agent goes several times on the same cell, it should be even more penalised, especially to prevent 
        #it to be stuck on 2 cells
        self.visited_cells[self.x, self.y] -= malus

        # add the feature corresponding to the visited cells to the state of the agent
        visited_cells_state =  self.visited_cells[self.x-2:self.x+3, self.y-2:self.y+3]
        visited_cells_state = visited_cells_state.reshape(5,5,1)
        state = np.concatenate((state,visited_cells_state), axis=2)
        
        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""
        # reset environement
        state = Environment.reset(self)
        
        malus = 0.4
        
        # reset malus
        self.visited_cells = np.zeros((self.grid_size, self.grid_size))
        self.visited_cells[self.x, self.y] = -malus

        visited_cells_state =  self.visited_cells[self.x-2:self.x+3, self.y-2:self.y+3]
        visited_cells_state = visited_cells_state.reshape(5,5,1)
        state = np.concatenate((state,visited_cells_state), axis=2)
        
        return state
    
## use those samples of code:
#In train explore:
#state, reward, game_over = env.act(action, train=True)

## In Environment exploring:
# You will have to change n_state to 3 because you will use one more layer!
#reward = 0
#if train:
#    reward = -self.malus_position[self.x, self.y]
#self.malus_position[self.x, self.y] = 0.1

#reward = reward + self.board[self.x, self.y]
# 3 "feature" states instead of 2
#state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
#                                self.board.reshape(self.grid_size, self.grid_size,1),
#                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [103]:
# Training
#env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
env = EnvironmentExploring()
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/030 | Loss 0.0000 | Win/lose count 4.0/12.0 (-8.0)
Epoch 001/030 | Loss 0.0000 | Win/lose count 2.5/6.0 (-3.5)
Epoch 002/030 | Loss 0.0000 | Win/lose count 6.5/6.0 (0.5)
Epoch 003/030 | Loss 0.0000 | Win/lose count 3.5/7.0 (-3.5)
Epoch 004/030 | Loss 0.0000 | Win/lose count 5.0/6.0 (-1.0)
Epoch 005/030 | Loss 0.0000 | Win/lose count 2.5/5.0 (-2.5)
Epoch 006/030 | Loss 0.0000 | Win/lose count 7.5/6.0 (1.5)
Epoch 007/030 | Loss 0.0000 | Win/lose count 2.0/7.0 (-5.0)
Epoch 008/030 | Loss 0.0000 | Win/lose count 6.5/8.0 (-1.5)
Epoch 009/030 | Loss 0.0000 | Win/lose count 4.5/3.0 (1.5)
Epoch 010/030 | Loss 0.0000 | Win/lose count 7.0/6.0 (1.0)
Epoch 011/030 | Loss 0.0000 | Win/lose count 4.0/7.0 (-3.0)
Epoch 012/030 | Loss 0.0000 | Win/lose count 4.0/7.0 (-3.0)
Epoch 013/030 | Loss 0.0000 | Win/lose count 3.5/4.0 (-0.5)
Epoch 014/030 | Loss 0.0000 | Win/lose count 3.0/3.0 (0.0)
Epoch 015/030 | Loss 0.0000 | Win/lose count 2.0/3.0 (-1.0)
Epoch 016/030 | Loss 0.0000 | Win/lose count

In [104]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore0.mp4'))

Epoch 000/010 | Loss 0.0000 | Win/lose count 2.0/0 (2.0)
Win/lose count 2.0/0. Average score (4.0)
Epoch 001/010 | Loss 0.0000 | Win/lose count 0.5/2.0 (-1.5)
Win/lose count 0.5/2.0. Average score (0.5)
Epoch 002/010 | Loss 0.0000 | Win/lose count 0.5/1.0 (-0.5)
Win/lose count 0.5/1.0. Average score (0.0)
Epoch 003/010 | Loss 0.0000 | Win/lose count 1.0/0 (1.0)
Win/lose count 1.0/0. Average score (0.5)
Epoch 004/010 | Loss 0.0000 | Win/lose count 1.5/1.0 (0.5)
Win/lose count 1.5/1.0. Average score (0.6)
Epoch 005/010 | Loss 0.0000 | Win/lose count 1.5/3.0 (-1.5)
Win/lose count 1.5/3.0. Average score (0.0)
Epoch 006/010 | Loss 0.0000 | Win/lose count 1.0/0 (1.0)
Win/lose count 1.0/0. Average score (0.2857142857142857)
Epoch 007/010 | Loss 0.0000 | Win/lose count 0.5/5.0 (-4.5)
Win/lose count 0.5/5.0. Average score (-0.875)
Epoch 008/010 | Loss 0.0000 | Win/lose count 0.5/5.0 (-4.5)
Win/lose count 0.5/5.0. Average score (-1.7777777777777777)
Epoch 009/010 | Loss 0.0000 | Win/lose count 0

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***